# data_d25a.ipynb
1. Time series: for global mean, relative, and geocentric at gauges, save fusion, low, central, high, and high-end projections for 2020–2100.
2. Year-2100: for relative sea level at gauges / near cities, save gauge/city info and projections for 2100.

Author: Benjamin S. Grandey.

In [1]:
import d25a
import datetime
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
# Get start datetime
start_dt = datetime.datetime.now()

In [3]:
# Print package versions
print(d25a.get_watermark())

Python implementation: CPython
Python version       : 3.10.16
IPython version      : 8.31.0

matplotlib: 3.10.0
numpy     : 2.2.2
pandas    : 2.2.3
seaborn   : 0.13.2
xarray    : 2025.1.1

conda environment: d25a-rsl-fusion

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 22.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



## 1. Time series of global mean, relative, and geocentric sea level for fusion, low, central, high, and high-end projections

In [4]:
# Loop over global mean, relative, and geocentric sea level
for slr_str in ('gmsl', 'rsl', 'novlm'):
    # Loop over fusion, low etc projections
    for proj_str in ('fusion-ssp585', 'fusion-ssp126', 'low', 'central', 'high', 'high-end'):
        # Get and write projection time series
        d25a.write_proj_ts_da(slr_str=slr_str, proj_str=proj_str)

Writing time_series/gmsl_fusion-ssp585_d25a.nc
Writing time_series/gmsl_fusion-ssp126_d25a.nc
Writing time_series/gmsl_low_d25a.nc
Writing time_series/gmsl_central_d25a.nc
Writing time_series/gmsl_high_d25a.nc
Writing time_series/gmsl_high-end_d25a.nc
Writing time_series/rsl_fusion-ssp585_d25a.nc (1843 locations)
Writing time_series/rsl_fusion-ssp126_d25a.nc (1843 locations)
Writing time_series/rsl_low_d25a.nc (1843 locations)
Writing time_series/rsl_central_d25a.nc (1843 locations)
Writing time_series/rsl_high_d25a.nc (1843 locations)
Writing time_series/rsl_high-end_d25a.nc (1843 locations)
Writing time_series/novlm_fusion-ssp585_d25a.nc (1843 locations)
Writing time_series/novlm_fusion-ssp126_d25a.nc (1843 locations)
Writing time_series/novlm_low_d25a.nc (1843 locations)
Writing time_series/novlm_central_d25a.nc (1843 locations)
Writing time_series/novlm_high_d25a.nc (1843 locations)
Writing time_series/novlm_high-end_d25a.nc (1843 locations)


### Save gauge information

In [5]:
# Create DataFrame to hold gauge information
gauge_info_df = pd.DataFrame(columns=['gauge_id', 'gauge_name', 'country', 'lat', 'lon'])
# Loop over locations for which projections are available
qfs_da = d25a.get_sl_qfs().copy()
for location in qfs_da.locations.data:
    gauge_info = d25a.get_gauge_info(location)
    gauge_info_df.loc[len(gauge_info_df)] = gauge_info
# Rename columns
gauge_info_df = gauge_info_df.rename(columns={'country': 'gauge_country', 'lat': 'gauge_lat', 'lon': 'gauge_lon'})
# Index by gauge_id
gauge_info_df = gauge_info_df.set_index('gauge_id')
# Save to CSV
out_fn = d25a.DATA_DIR / 'time_series' / 'gauge_info_d25a.csv'
print(f'Writing {out_fn.name} ({len(gauge_info_df)} gauges)')
gauge_info_df.to_csv(out_fn)
gauge_info_df.head()

Writing gauge_info_d25a.csv (1843 gauges)


,gauge_name,gauge_country,gauge_lat,gauge_lon
gauge_id,,,,
1,BREST,FRANCE,48.38,-4.49
2,SWINOUJSCIE,POLAND,53.92,14.23
3,SHEERNESS,UNITED KINGDOM,51.45,0.74
5,HOLYHEAD,UNITED KINGDOM,53.31,-4.62
7,CUXHAVEN_2,GERMANY,53.87,8.72


## 2. Year-2100 projections at gauges / near cities, with corresponding gauge/city information

In [6]:
out_dir = d25a.DATA_DIR / 'year_2100'
if not out_dir.exists():
    out_dir.mkdir()

### 2a. Gauges

In [7]:
# Copy gauge info
gauges_df = gauge_info_df.copy()
# Get low-end, central, and high-end rsl and novlm projections for 2100
for rsl_novlm in ('rsl', 'novlm'):  # loop over rsl and novlm
    for low_central_high in ('low', 'central', 'high'):  # loop over low-end, central, and high-end projections
        col = f'{rsl_novlm}_{low_central_high}'
        proj_da = xr.open_dataset(d25a.DATA_DIR / 'time_series' / f'{rsl_novlm}_{low_central_high}_d25a.nc'
                                  )['sea_level_change'].sel(years=2100)  # get year-2100 projections at gauges
        proj_da = proj_da.round(2)  # round to nearest cm
        for index, row_ser in gauges_df.iterrows():  # loop over gauges and save projection to gauges_df
            gauges_df.loc[index, col] = float(proj_da.sel(locations=index))
# Save to CSV
out_fn = out_dir / f'gauges_2100_d25a.csv'
print(f'Writing {out_fn.name} ({len(gauges_df)} gauges)')
gauges_df.to_csv(out_fn)
gauges_df.head(30)

Writing gauges_2100_d25a.csv (1843 gauges)


,gauge_name,gauge_country,gauge_lat,gauge_lon,rsl_low,rsl_central,rsl_high,novlm_low,novlm_central,novlm_high
gauge_id,,,,,,,,,,
1,BREST,FRANCE,48.38,-4.49,0.29,0.54,0.94,0.30,0.55,0.95
2,SWINOUJSCIE,POLAND,53.92,14.23,0.29,0.58,1.01,0.32,0.61,1.04
3,SHEERNESS,UNITED KINGDOM,51.45,0.74,0.32,0.61,1.03,0.29,0.58,1.00
5,HOLYHEAD,UNITED KINGDOM,53.31,-4.62,0.29,0.54,0.93,0.28,0.54,0.93
7,CUXHAVEN_2,GERMANY,53.87,8.72,0.34,0.64,1.07,0.28,0.57,1.00
8,WISMAR_2,GERMANY,53.90,11.46,0.26,0.64,1.14,0.25,0.63,1.13
9,MAASSLUIS,"NETHERLANDS, KINGDOM OF THE",51.92,4.25,0.31,0.60,1.03,0.28,0.57,1.00
10,SAN_FRANCISCO,UNITED STATES,37.81,-122.47,0.32,0.53,0.91,0.30,0.50,0.89
11,WARNEMUNDE_2,GERMANY,54.17,12.10,0.30,0.59,1.02,0.32,0.60,1.03


### 2b. All cities
The cities are urban agglomerations with a population of at least 300,000 inhabitants in 2018, according to the UN's World Urbanization Prospects (2018).

Low-end, central, and high-end RSL projections for 2100 are saved if the distance to the nearest available tide gauge is ≤ 100km.

In [8]:
# Read World Urbanisation Prospects 2018 data
cities_df = pd.read_excel('data_in/wup18/WUP2018-F12-Cities_Over_300K.xls', header=16, usecols='A,C,E,G,H,X', index_col=None)
cities_df = cities_df.rename(columns={'Index': 'city_index', 'Country or area': 'city_country', 'Urban Agglomeration': 'city_name',
                                      'Latitude': 'city_lat', 'Longitude': 'city_lon', 2025: 'population_2025_1000s'})
# Reorder columns
cities_df = cities_df.set_index('city_index')
cities_df = cities_df[['city_name', 'city_country', 'city_lat', 'city_lon', 'population_2025_1000s']]
# Loop over these cities and get data for nearest tide gauge
for index, row_ser in cities_df.iterrows():
    lat0 = row_ser['city_lat']  # latitude of city
    lon0 = row_ser['city_lon']  # longitude of city
    temp_df = gauges_df.copy()  # copy tide gauge data (from above)
    temp_df['distance_km'] = 6378 * np.arccos(  # calculate great-circle distance between city and all available gauges
        np.sin(np.radians(lat0)) * np.sin(np.radians(temp_df['gauge_lat'])) +
        np.cos(np.radians(lat0)) * np.cos(np.radians(temp_df['gauge_lat'])) * np.cos(np.radians(temp_df['gauge_lon'] - lon0)))
    temp_df = temp_df.sort_values(by=['distance_km']).reset_index()  # sort by distance
    for col in ['gauge_id', 'gauge_name', 'gauge_lat', 'gauge_lon', 'distance_km']:
        cities_df.loc[index, col] = temp_df.loc[0, col]  # save gauge info to cities_df
    if temp_df.loc[0, 'distance_km'] < 100.5:  # if distance to nearest gauge is ≤ 100km...
        for rsl_novlm in ('rsl', 'novlm'):  # ...save projections for year-2100
            for low_central_high in ('low', 'central', 'high'):
                col = f'{rsl_novlm}_{low_central_high}'
                cities_df.loc[index, col] = temp_df.loc[0, col]
# Rounding
for col in ['city_lat', 'city_lon']:  # round to 2 d.p.
    cities_df[col] = cities_df[col].round(2)
for col in ['population_2025_1000s', 'gauge_id', 'distance_km']:  # round to nearest integer
    cities_df[col] = cities_df[col].round(0).astype(int)
# Save to CSV
out_fn = out_dir / f'cities_2100_d25a.csv'
print(f'Writing {out_fn.name} ({len(cities_df)} cities; {len(cities_df.dropna())} within 100km of gauge)')
cities_df.to_csv(out_fn)
cities_df.head(30)

Writing cities_2100_d25a.csv (1860 cities; 1193 within 100km of gauge)


,city_name,city_country,city_lat,city_lon,population_2025_1000s,gauge_id,gauge_name,gauge_lat,gauge_lon,distance_km,rsl_low,rsl_central,rsl_high,novlm_low,novlm_central,novlm_high
city_index,,,,,,,,,,,,,,,,
1,Herat,Afghanistan,34.35,62.20,753,1005300540,grid_37.0_54.0,37.00,54.00,798,NaN,NaN,NaN,NaN,NaN,NaN
2,Kabul,Afghanistan,34.53,69.17,4877,1006200680,grid_28.0_68.0,28.00,68.00,735,NaN,NaN,NaN,NaN,NaN,NaN
3,Kandahar,Afghanistan,31.61,65.71,577,1006200680,grid_28.0_68.0,28.00,68.00,459,NaN,NaN,NaN,NaN,NaN,NaN
4,Mazar-e Sharif,Afghanistan,36.71,67.11,682,1006200680,grid_28.0_68.0,28.00,68.00,973,NaN,NaN,NaN,NaN,NaN,NaN
5,Tiranë (Tirana),Albania,41.33,19.82,536,1004900200,grid_41.0_20.0,41.00,20.00,39,0.23,0.52,0.94,0.28,0.53,0.92
6,Annaba,Algeria,36.90,7.77,379,1005300080,grid_37.0_8.0,37.00,8.00,24,0.26,0.56,0.99,0.31,0.56,0.96
7,Batna,Algeria,35.56,6.17,358,1005400060,grid_36.0_6.0,36.00,6.00,52,0.25,0.54,0.98,0.31,0.56,0.96
8,Blida,Algeria,36.48,2.83,536,1005400030,grid_36.0_3.0,36.00,3.00,56,0.27,0.55,0.98,0.32,0.56,0.96
9,El Djazaïr (Algiers),Algeria,36.75,3.04,3004,1005300030,grid_37.0_3.0,37.00,3.00,28,0.29,0.57,1.00,0.32,0.56,0.96


### 2c. Megacities
Select subset of cities with population ≥ 10 million in 2025.

In [9]:
# Select cities with a population ≥ 10,000 thousand
mega_df = cities_df.loc[cities_df['population_2025_1000s'] >= 10000].sort_values(by='population_2025_1000s', ascending=False)
# Identify shorter name for megacities with a long name
for index, row_ser in mega_df.iterrows():
    short_name = row_ser['city_name']  # use full name by default
    if short_name.split(' (')[0] in ['Mumbai', 'Kolkata', 'Chennai']:  # cases to use name outside parentheses
        short_name = short_name.split(' (')[0]
    elif '(' in short_name:  # cases to use name within parentheses
        short_name = short_name.split(' (')[-1].rstrip(')')
    elif ',' in short_name:  # cases to use name before comma
        short_name = short_name.split(',')[0]
    elif '-' in short_name:  # cases to use name before hyphen
        short_name = short_name.split('-')[0]
    mega_df.loc[index, 'city_short'] = short_name
# Save to CSV
out_fn = out_dir / f'megacities_2100_d25a.csv'
print(f'Writing {out_fn.name} ({len(mega_df)} megacities; {len(mega_df.dropna())} within 100km of gauge)')
mega_df.to_csv(out_fn)
mega_df

Writing megacities_2100_d25a.csv (37 megacities; 31 within 100km of gauge)


,city_name,city_country,city_lat,city_lon,population_2025_1000s,gauge_id,gauge_name,gauge_lat,gauge_lon,distance_km,rsl_low,rsl_central,rsl_high,novlm_low,novlm_central,novlm_high,city_short
city_index,,,,,,,,,,,,,,,,,
1088,Tokyo,Japan,35.69,139.69,37036,1545,TOKYO_III,35.65,139.77,8,0.17,0.45,0.93,0.35,0.62,1.11,Tokyo
808,Delhi,India,28.67,77.22,34666,1006700730,grid_23.0_73.0,23.00,73.00,759,NaN,NaN,NaN,NaN,NaN,NaN,Delhi
449,Shanghai,China,31.22,121.46,30482,1005901210,grid_31.0_121.0,31.00,121.00,50,0.14,0.54,1.12,0.33,0.58,1.02,Shanghai
61,Dhaka,Bangladesh,23.71,90.41,24653,1006600900,grid_24.0_90.0,24.00,90.00,53,0.09,0.48,1.01,0.28,0.50,0.91,Dhaka
685,Al-Qahirah (Cairo),Egypt,30.04,31.24,23074,1006000310,grid_30.0_31.0,30.00,31.00,23,0.24,0.54,1.00,0.30,0.56,0.99,Cairo
138,São Paulo,Brazil,-23.55,-46.64,22990,1011403130,grid_-24.0_313.0,-24.00,-47.00,63,0.40,0.66,1.06,0.32,0.56,0.95,São Paulo
1147,Ciudad de México (Mexico City),Mexico,19.43,-99.14,22752,1007102610,grid_19.0_261.0,19.00,-99.00,50,0.52,0.79,1.20,0.34,0.56,0.95,Mexico City
204,Beijing,China,39.91,116.40,22596,1005001160,grid_40.0_116.0,40.00,116.00,35,0.08,0.49,1.06,0.27,0.52,0.96,Beijing
882,Mumbai (Bombay),India,19.07,72.88,22089,1007100730,grid_19.0_73.0,19.00,73.00,15,0.09,0.49,1.05,0.28,0.51,0.95,Mumbai


In [10]:
# Get end datetime
end_dt = datetime.datetime.now()
# Calculate run timedelta
run_td = end_dt - start_dt
# Print timing information
print(f"Start:     {start_dt.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"End:       {end_dt.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Wall time: {run_td.seconds} s")

Start:     2025-04-29 14:43:31
End:       2025-04-29 14:45:00
Wall time: 89 s
